In [1]:
import pandas as pd
import numpy as np
from astropy.io import ascii
from astropy.table import QTable, Table, Column
from astropy import units as u

In [2]:
rrlyr = pd.read_table("rrlyr_vsx_clean.txt")
rrlyr

,RRAB 0.01307 -2.43007 PS1-3PI J000003.12-022548.2
0,RRAB 0.01521 +35.36286 GM And
1,RRAB 0.01669 +18.40698 CSS_J000004.0+182425
2,RRAB 0.02771 +56.65331 V1023 Cas
3,RRAB 0.03171 +34.67397 SERIV 27
4,RRAB 0.03901 +5.58986 CSS_J000009.3+053523
...,...
145148,RRC 359.96926 -7.01093 PS1-3PI J235952.62-0700...
145149,RRAB 359.97391 +17.33493 PS1-3PI J235953.72+17...
145150,RRAB 359.97879 -24.95662 PS1-3PI J235954.90-24...
145151,RRC 359.99107 -7.45278 PS1-3PI J235957.85-0727...


In [3]:
column1 = []
column2 = []
column3 = []
column4 = []

with open('rrlyr_vsx_clean.txt', 'r') as file:
    for line in file:
        empty = []
        components = line.split(' ', 3)
    
        column1.append(components[0])
        column2.append(float(components[1]))
        column3.append(float(components[2]))
        column4.append(components[3].strip())

# column2 = column2 * u.deg
# column3 = column3 * u.deg

final_table = QTable([column1, column2, column3, column4], names = ("Star Type", "RA", "Dec", "Other Info"))

In [4]:
final_table

Star Type,RA,Dec,Other Info
str24,float64,float64,str30
RRAB,0.01307,-2.43007,PS1-3PI J000003.12-022548.2
RRAB,0.01521,35.36286,GM And
RRAB,0.01669,18.40698,CSS_J000004.0+182425
RRAB,0.02771,56.65331,V1023 Cas
RRAB,0.03171,34.67397,SERIV 27
RRAB,0.03901,5.58986,CSS_J000009.3+053523
RRAB,0.04274,-2.51629,PS1-3PI J000010.25-023058.6
RRAB,0.04331,-21.92086,CSS 120828:000010-215515
RRAB,0.04782,62.71524,Gaia DR2 430046902041301376


In [5]:
from pyasassn.client import SkyPatrolClient	
client = SkyPatrolClient()

Welcome to ASAS-SN Skypatrol!

Current Deployment Version: 0.6.17 (26 JAN 2024)
Please upgrade your client if not up to date.



In [6]:
final_table["RA"][0]

0.01307

In [7]:
# search = client.cone_search(ra_deg=270, dec_deg=88, radius=0.1, catalog='master_list', download=True)
search = client.cone_search(ra_deg = final_table["RA"][1], dec_deg = final_table["Dec"][1], radius=0.0003, catalog='master_list', download=True)
print(search)


# print(search[8590476870].data)

path = search.save(save_dir='/Users/zoesurles/Desktop/variable_star_classification/downloaded_lightcurves/', file_format="csv")[1]

Pulled 1 of 1
LightCurveCollection with 1 light curves 
   asas_sn_id    ra_deg    dec_deg                catalog_sources
0  8590948612  0.015352  35.362782  [stellar_main, tic, aavsovsx]
